In [ ]:
%tensorflow_version 1.x
!apt-get install ffmpeg freeglut3-dev xvfb
!pip install stable-baselines[mpi]==2.10.0

!git clone -l -s git://github.com/wjwswain/chaRL.git chaRL
%cd chaRL/games/blackjack
from env import GameEnv

import gym
import numpy as np
from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines import A2C

TensorFlow 1.x selected.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
freeglut3-dev set to manually installed.
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 784 kB of archives.
After this operation, 2,266 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.4 [784 kB]
Fetched 784 kB in 1s (586 kB/s)
Selecting previously unselected package xvfb.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.4_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.4) ..

In [ ]:
from stable_baselines import DQN

env = GameEnv(5000,"Wesley")
model = DQN(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=25000)
model.save("dqn_mlp_blackjack")

while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

In [22]:
#https://dev.to/nexttech/build-a-blackjack-command-line-game-3o4b
#https://stable-baselines.readthedocs.io/en/master/guide/custom_env.html

import numpy as np
import random
import gym
from gym import spaces

class GameEnv(gym.Env):
	metadata = {'render.modes': ['console']}

	def __init__(self, bankroll, name):
		super(GameEnv, self).__init__()

		self.bet = 0
		self.rounds = 1
		self.bankroll = bankroll
		self.name = name

		self.deck = Deck()
		self.deck.shuffle()
		self.hand = Hand()
		self.dealer = Hand()

		self.observation = np.zeros(157, dtype=np.int8)#[stage(0), deck left(1-52), player's hand(53-104), dealer's hand(105-156)]
		self.observation[1:53] = 1
		self.reward = 0
		self.done = False
		self.info = {}

		self.action_space = spaces.MultiDiscrete([100,2])# [$5-$500 bid, hit(0)/stay(1)]
		self.observation_space = spaces.MultiBinary(157)# I hate the limitations of stable-baselines

	def reshuffle(self):
		self.deck = Deck()
		self.deck.shuffle()
		self.observation[1:53] = 1 #deck left

	def step(self, action):
		if self.observation[0] == 0: #stage
			if self.deck.len() < 3:
				self.reshuffle()

			self.rounds += 1
			self.bet = min(5*(action[0]+1), self.bankroll)
			self.bankroll -= self.bet

			self.hand = Hand()
			for hand_i in range(2):
				idx = self.hand.add_card(self.deck.deal())
				self.observation[1+idx] = 0
				self.observation[53+idx] = 1

			self.dealer = Hand()
			idx = self.dealer.add_card(self.deck.deal())
			self.observation[1+idx] = 0
			self.observation[105+idx] = 1

			self.observation[0] = 1
			self.reward = 0

		elif action[1] == 0:
			if self.deck.len() == 0:
				self.reshuffle()
			idx = self.hand.add_card(self.deck.deal())
			self.observation[1+idx] = 0
			self.observation[53+idx] = 1

			if self.hand.value[-1] > 21:
				self.reward = -self.bet
				self.observation[0] = 0
				self.observation[53:] = 0
				if self.rounds == 500 or int(self.bankroll) < 5:
					self.done = True
			else:
				self.reward = 0

		else:
			while self.dealer.value[0] < 17:
				if self.deck.len() == 0:
					self.reshuffle()
				idx = self.dealer.add_card(self.deck.deal())

			for hand in [self.hand, self.dealer]:
				if len(hand.value) > 1:
					hand.best_val = 21 - min([21-val for val in hand.value])
				else:
					hand.best_val = hand.value[0]
			if self.hand.best_val == self.dealer.best_val and self.hand.best_val < 22:
				self.reward = self.bet
				self.bankroll += self.reward
			elif self.hand.best_val > self.dealer.best_val and self.hand.best_val < 22:
				self.reward = 1.5*self.bet
				self.bankroll += self.reward
			else:
				self.reward = -self.bet

			self.observation[0] = 0
			self.observation[53:] = 0

			if self.rounds == 500 or int(self.bankroll) < 5:
				self.done = True
		return self.observation, self.reward, self.done, self.info

	def reset(self):
		self.deck = Deck()
		self.deck.shuffle()

		self.observation = np.zeros(157, dtype=np.int8)
		self.observation[1:53] = 1

		return self.observation

	def render(self, mode="console"):
		if mode != "console":
			raise NotImplementedError()
		else:
			if self.observation[0] == 0:
				if self.rounds > 1:
					print("Round Reward:", self.reward)
					print()
				print(self.name + ': ' + str(self.bankroll))
			else:
				print(self.name, "bet", str(self.bet) + ':')
				if len(self.hand.cards) > 0:
					self.hand.display()
					print()
					print("Dealer:")
					print("??")
					self.dealer.display()
					print()

	def close (self):
		pass


class Deck:
	def __init__(self):
		values = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
		suits = ['C', 'H', 'S', 'D']
		self.cards = [Card(v,s) for v in values for s in suits]

	def shuffle(self):
		random.shuffle(self.cards)

	def deal(self):
		return self.cards.pop(0)

	def len(self):
		return len(self.cards)


class Hand:
	def __init__(self):
		self.cards = []
		self.value = [0]
		self.best_val = 0

	def display(self):
		for card in self.cards:
			print(card)

	def add_card(self, card):
		if card.isAce():
			self.value.append(-10)
		self.cards.append(card)
		for i in range(len(self.value)):
			self.value[i] += card.get_value()
		return card.get_index()


class Card:
	def __init__(self, value, suit):
		self.value = value
		self.suit = suit

	def __repr__(self):
		return self.value + self.suit

	def get_value(self):
		if self.value.isnumeric():
			return int(self.value)
		elif self.isAce():
			return 11
		else:
			return 10

	def get_index(self):
		suits = {'C':0, 'H':1, 'S':2, 'D':3}
		faces = {'A':0, 'J':10, 'Q':11, 'K':12}
		suit_index = suits[self.suit]
		if self.value.isnumeric():
			value_index = int(self.value)
		else:
			value_index = faces[self.value]
		return (4 * value_index) + suit_index

	def isAce(self):
		return self.value == 'A'


In [25]:
# !git clone -l -s git://github.com/wjwswain/chaRL.git chaRL
# %cd chaRL/games/blackjack
# from env import GameEnv

env = GameEnv(5000,"Wesley")
env.reset()
for _ in range(100):
    env.render()
    o,r,d,i = env.step(env.action_space.sample())
    if d:
      break
env.close()

Wesley: 5000
Wesley bet 30:
4C
3S

Dealer:
??
6H

Round Reward: -30

Wesley: 4970
Wesley bet 80:
3C
KS

Dealer:
??
8H

Wesley bet 80:
3C
KS
2S

Dealer:
??
8H

Round Reward: -80

Wesley: 4890
Wesley bet 385:
6D
AH

Dealer:
??
3H

Wesley bet 385:
6D
AH
JC

Dealer:
??
3H

Round Reward: -385

Wesley: 4505
Wesley bet 450:
AS
KD

Dealer:
??
7H

Round Reward: 675.0

Wesley: 4730.0
Wesley bet 220:
9H
2D

Dealer:
??
2C

Round Reward: -220

Wesley: 4510.0
Wesley bet 205:
JD
5D

Dealer:
??
QS

Round Reward: -205

Wesley: 4305.0
Wesley bet 335:
KC
9S

Dealer:
??
10S

Round Reward: -335

Wesley: 3970.0
Wesley bet 390:
10C
5H

Dealer:
??
10D

Round Reward: -390

Wesley: 3580.0
Wesley bet 300:
6C
6S

Dealer:
??
2H

Round Reward: -300

Wesley: 3280.0
Wesley bet 235:
4D
4S

Dealer:
??
9C

Round Reward: -235

Wesley: 3045.0
Wesley bet 500:
4H
7C

Dealer:
??
7S

Round Reward: -500

Wesley: 2545.0
Wesley bet 170:
9S
2C

Dealer:
??
8C

Wesley bet 170:
9S
2C
10S

Dealer:
??
8C

Round Reward: -170

Wesley: 2